In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [2]:
!ls "/gdrive/My Drive/bigcontest2019/data/" # address of our files

test  train


In [0]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

matplotlib.rcParams['figure.figsize'] = (15, 8)

from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# model
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge

In [4]:
path = '/gdrive/My Drive/bigcontest2019/data/'
print(os.listdir(path + 'train'),'\n')

['train_payment.csv', 'train_trade.csv', 'train_pledge.csv', 'train_combat.csv', 'train_label.csv', 'train_activity.csv', 'train_merge_bjw.csv'] 



In [0]:
train_label = pd.read_csv(path+'train/'+'train_label.csv')
train_activity = pd.read_csv(path+'train/'+'train_activity.csv')

In [6]:
train_label['survival_time']<8

0        False
1        False
2        False
3        False
4        False
5        False
6        False
7         True
8        False
9        False
10       False
11       False
12       False
13       False
14        True
15       False
16       False
17       False
18       False
19       False
20       False
21       False
22       False
23       False
24       False
25       False
26       False
27        True
28       False
29       False
         ...  
39970    False
39971     True
39972    False
39973    False
39974    False
39975    False
39976    False
39977     True
39978    False
39979    False
39980    False
39981    False
39982    False
39983    False
39984    False
39985     True
39986    False
39987    False
39988    False
39989    False
39990    False
39991    False
39992    False
39993    False
39994    False
39995    False
39996    False
39997    False
39998    False
39999    False
Name: survival_time, Length: 40000, dtype: bool

In [0]:
train_activity['game_money_change'] = np.abs(train_activity['game_money_change'])
train_activity_group = train_activity.groupby('acc_id').agg({'day':'nunique', 
                                                             'char_id':'nunique', 
                                                             'server':'nunique', 
                                                             'playtime':['sum','mean'], 
                                                             'npc_kill':['sum','mean'], 
                                                             'solo_exp':'sum',
                                                             'party_exp':'sum', 
                                                             'quest_exp':'sum',
                                                             'rich_monster':'sum', 
                                                             'death':'sum', 
                                                             'revive':'sum',
                                                             'exp_recovery':'sum',
                                                             'fishing':'sum',
                                                             'private_shop':'sum',
                                                             'game_money_change':'sum',
                                                             'enchant_count':'sum'})

In [0]:
train_activity_group.columns = list(map('_'.join, train_activity_group.columns.values))

In [0]:
train_activity_grouped = train_activity_group.reset_index()

In [10]:
train_activity_grouped.columns

Index(['acc_id', 'day_nunique', 'char_id_nunique', 'server_nunique',
       'playtime_sum', 'playtime_mean', 'npc_kill_sum', 'npc_kill_mean',
       'solo_exp_sum', 'party_exp_sum', 'quest_exp_sum', 'rich_monster_sum',
       'death_sum', 'revive_sum', 'exp_recovery_sum', 'fishing_sum',
       'private_shop_sum', 'game_money_change_sum', 'enchant_count_sum'],
      dtype='object')

In [11]:
train_activity_grouped.head()

,acc_id,day_nunique,char_id_nunique,server_nunique,playtime_sum,playtime_mean,npc_kill_sum,npc_kill_mean,solo_exp_sum,party_exp_sum,quest_exp_sum,rich_monster_sum,death_sum,revive_sum,exp_recovery_sum,fishing_sum,private_shop_sum,game_money_change_sum,enchant_count_sum
0,2,28,1,1,91.533663,3.269059,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.0,0.000000,84.900753,4.056205,0.0
1,5,10,3,1,6.116132,0.339785,3.236452,0.179803,2.155434,0.695285,0.111055,9,0.245883,0.247337,0.0,3.442909,0.000000,0.328911,0.0
2,8,28,7,1,18.797802,0.141337,31.172609,0.234381,31.489136,0.000000,1.561055,2,91.960416,92.504000,0.0,4.807823,0.000000,9.238591,0.0
3,17,28,4,1,28.464708,0.547398,30.456852,0.585709,9.255848,8.823094,2.322365,21,1.721184,1.731358,0.0,13.990954,0.000000,6.126588,0.0
4,20,27,2,4,25.588043,0.673370,34.100184,0.897373,0.274902,0.080060,0.083483,8,0.983534,0.989348,0.0,4.295766,2.917510,8.536558,0.0


In [0]:
SEED = 1234

In [0]:
skf = StratifiedKFold(n_splits=5, random_state=SEED, shuffle=True)
for trn_idx, val_idx in skf.split(train_label, y=train_label['survival_time']):
    train_df = train_activity_grouped.loc[trn_idx]
    valid_df = train_activity_grouped.loc[val_idx]
    break

In [0]:
train_dfl = pd.merge(train_df, train_label, how='inner', on='acc_id')
valid_dfl = pd.merge(valid_df, train_label, how='inner', on='acc_id')

## survival_time

In [0]:
FOLDS = 5
lalabels = ['survival_time', 'amount_spent']

In [0]:
train_dfls = train_dfl.drop(['amount_spent'], axis=1)
valid_dfls = valid_dfl.drop(['amount_spent'], axis=1)
lalabel = lalabels[0]

In [0]:
class model(object):
    def __init__(self, train, test, folds=FOLDS, seed=SEED, log=True):
        self.train = train
        self.test = test.drop(lalabel, axis=1)
        self.kf = KFold(n_splits=folds, random_state=seed)
        self.log = log
    
    def ridge_model(self):
        oof = np.zeros(len(self.train))
        pred = np.zeros(len(self.test))
        for trn_idx, val_idx in self.kf.split(self.train):
            train_df = self.train.loc[trn_idx]
            valid_df = self.train.loc[val_idx].drop(lalabel, axis=1)
            if self.log:
                ridge_model = Ridge().fit(train_df.drop(lalabel, axis=1), np.log1p(train_df[lalabel]))
            else:
                ridge_model = Ridge().fit(train_df.drop(lalabel, axis=1), train_df[lalabel])
            oof[val_idx] = ridge_model.predict(valid_df)
            pred += ridge_model.predict(self.test)/self.kf.n_splits
        return oof, pred
    
    
    def rf_model(self):
        
    
        
        return oof, pred
    
    
    def xgb_model(self):
        warnings.filterwarnings(action='ignore')
        
        params={
            'eta': 0.001,
            'max_depth': 16,
            'min_child_weight': 16,
            'n_estimators' : 10000,
            'subsample': 0.9
        }
    
    
        oof = np.zeros(len(self.train))
        pred = np.zeros(len(self.test))
        for trn_idx, val_idx in self.kf.split(self.train):
            if self.log:
                train_df = xgb.DMatrix(self.train.loc[trn_idx].drop(lalabel, axis=1), label=np.log1p(self.train.loc[trn_idx, lalabel]))
                valid_df = xgb.DMatrix(self.train.loc[val_idx].drop(lalabel, axis=1), label=np.log1p(self.train.loc[val_idx, lalabel]))
            else:
                train_df = xgb.DMatrix(self.train.loc[trn_idx].drop(lalabel, axis=1), label=self.train.loc[trn_idx, lalabel])
                valid_df = xgb.DMatrix(self.train.loc[val_idx].drop(lalabel, axis=1), label=self.train.loc[val_idx, lalabel])
            xgb_model = xgb.train(params, train_df, num_boost_round=30000, evals=[(train_df, 'train'), (valid_df, 'val')], verbose_eval=5000, early_stopping_rounds=500)
            oof[val_idx] = xgb_model.predict(xgb.DMatrix(self.train.loc[val_idx].drop(lalabel, axis=1)))
            pred += xgb_model.predict(xgb.DMatrix(self.test))/self.kf.n_splits
        return oof, pred
    
    def rmse(self, true, pred):
        
        if self.log:
            true = np.expm1(pred)
            mse = mean_squared_error(true, pred)
            rmse = np.round(np.sqrt(mse), 2)
        else:
            mse = mean_squared_error(true, pred)
            rmse = np.round(np.sqrt(mse), 2)
    
        return rmse

In [19]:
train_dfls.head()

,acc_id,day_nunique,char_id_nunique,server_nunique,playtime_sum,playtime_mean,npc_kill_sum,npc_kill_mean,solo_exp_sum,party_exp_sum,quest_exp_sum,rich_monster_sum,death_sum,revive_sum,exp_recovery_sum,fishing_sum,private_shop_sum,game_money_change_sum,enchant_count_sum,survival_time
0,2,28,1,1,91.533663,3.269059,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.0,0.000000,84.900753,4.056205,0.0,64
1,5,10,3,1,6.116132,0.339785,3.236452,0.179803,2.155434,0.695285,0.111055,9,0.245883,0.247337,0.0,3.442909,0.000000,0.328911,0.0,60
2,17,28,4,1,28.464708,0.547398,30.456852,0.585709,9.255848,8.823094,2.322365,21,1.721184,1.731358,0.0,13.990954,0.000000,6.126588,0.0,64
3,20,27,2,4,25.588043,0.673370,34.100184,0.897373,0.274902,0.080060,0.083483,8,0.983534,0.989348,0.0,4.295766,2.917510,8.536558,0.0,64
4,21,26,2,2,21.831291,0.779689,10.771165,0.384684,20.064853,0.380025,9.524346,3,0.000000,0.000000,0.0,14.846620,0.000000,2.132342,0.0,64


In [20]:
valid_dfls.head()

,acc_id,day_nunique,char_id_nunique,server_nunique,playtime_sum,playtime_mean,npc_kill_sum,npc_kill_mean,solo_exp_sum,party_exp_sum,quest_exp_sum,rich_monster_sum,death_sum,revive_sum,exp_recovery_sum,fishing_sum,private_shop_sum,game_money_change_sum,enchant_count_sum,survival_time
0,8,28,7,1,18.797802,0.141337,31.172609,0.234381,31.489136,0.000000,1.561055,2,91.960416,92.504000,0.000000,4.807823,0.000000,9.238591,0.0,64
1,31,28,2,1,82.082097,2.160055,65.598568,1.726278,0.322670,0.006899,0.001952,0,1.475301,0.000000,0.000000,0.000000,0.000000,1.425392,0.0,45
2,69,27,6,1,23.219300,0.293915,37.755682,0.477920,0.415315,0.060528,0.028414,3,2.212951,1.731358,15.948733,4.273373,0.000000,0.687367,0.0,64
3,97,28,1,1,91.079576,3.252842,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,88.036763,3.774405,0.0,64
4,138,24,5,2,6.490637,0.141101,1.654555,0.035969,27.724019,0.058379,2.113416,0,1.721184,1.731358,15.948733,1.272224,0.043107,0.369373,0.0,37


In [21]:
mdl = model(train_dfls, valid_dfls)
mdl

In [22]:
valid_dfls.shape

(8024, 20)

In [23]:
mdl.ridge_model()[1].shape

(8024,)

In [0]:
pred_s = mdl.ridge_model()[1]

In [0]:
train_dfla = train_dfl.drop(['survival_time'], axis=1)
valid_dfla = valid_dfl.drop(['survival_time'], axis=1)
lalabel = lalabels[1]

In [26]:
train_dfla.head()

,acc_id,day_nunique,char_id_nunique,server_nunique,playtime_sum,playtime_mean,npc_kill_sum,npc_kill_mean,solo_exp_sum,party_exp_sum,quest_exp_sum,rich_monster_sum,death_sum,revive_sum,exp_recovery_sum,fishing_sum,private_shop_sum,game_money_change_sum,enchant_count_sum,amount_spent
0,2,28,1,1,91.533663,3.269059,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.0,0.000000,84.900753,4.056205,0.0,0.000000
1,5,10,3,1,6.116132,0.339785,3.236452,0.179803,2.155434,0.695285,0.111055,9,0.245883,0.247337,0.0,3.442909,0.000000,0.328911,0.0,0.000000
2,17,28,4,1,28.464708,0.547398,30.456852,0.585709,9.255848,8.823094,2.322365,21,1.721184,1.731358,0.0,13.990954,0.000000,6.126588,0.0,0.070642
3,20,27,2,4,25.588043,0.673370,34.100184,0.897373,0.274902,0.080060,0.083483,8,0.983534,0.989348,0.0,4.295766,2.917510,8.536558,0.0,0.052137
4,21,26,2,2,21.831291,0.779689,10.771165,0.384684,20.064853,0.380025,9.524346,3,0.000000,0.000000,0.0,14.846620,0.000000,2.132342,0.0,0.184267


In [27]:
valid_dfla.head()

,acc_id,day_nunique,char_id_nunique,server_nunique,playtime_sum,playtime_mean,npc_kill_sum,npc_kill_mean,solo_exp_sum,party_exp_sum,quest_exp_sum,rich_monster_sum,death_sum,revive_sum,exp_recovery_sum,fishing_sum,private_shop_sum,game_money_change_sum,enchant_count_sum,amount_spent
0,8,28,7,1,18.797802,0.141337,31.172609,0.234381,31.489136,0.000000,1.561055,2,91.960416,92.504000,0.000000,4.807823,0.000000,9.238591,0.0,0.020310
1,31,28,2,1,82.082097,2.160055,65.598568,1.726278,0.322670,0.006899,0.001952,0,1.475301,0.000000,0.000000,0.000000,0.000000,1.425392,0.0,0.051316
2,69,27,6,1,23.219300,0.293915,37.755682,0.477920,0.415315,0.060528,0.028414,3,2.212951,1.731358,15.948733,4.273373,0.000000,0.687367,0.0,0.030449
3,97,28,1,1,91.079576,3.252842,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,88.036763,3.774405,0.0,0.000000
4,138,24,5,2,6.490637,0.141101,1.654555,0.035969,27.724019,0.058379,2.113416,0,1.721184,1.731358,15.948733,1.272224,0.043107,0.369373,0.0,0.351767


In [28]:
mdl = model(train_dfla, valid_dfla)
mdl

In [29]:
mdl.ridge_model()[1].shape

(8024,)

In [0]:
pred_a = mdl.ridge_model()[1]

In [0]:
# -*- coding: utf-8 -*-

# ------------------------------------------------------------------------ #
# INPUT
#    predicted_label : 예측 답안지 파일 경로 
#    actual_label    : 실제 답안지 파일 경로
#
# OUTPUT             : 유저 기대이익 총합 
#
#
# - 예측 답안지를 실제 답안과 비교하여 유저 기대이익 총합을 계산하는 함수
# - 함수의 계산방식은 문제 설명서에 기술된 기대이익 산출식과 동일
# ------------------------------------------------------------------------ #

#필요한 모듈 import
import pandas as pd
import numpy as np
import sys

def score_function(predict_label, actual_label):
    
    predict = predict_label
    actual = actual_label

    predict.acc_id = predict.acc_id.astype('int')
    predict = predict.sort_values(by =['acc_id'], axis = 0) # 예측 답안을 acc_id 기준으로 정렬 
    predict = predict.reset_index(drop = True)
    actual.acc_id = actual.acc_id.astype('int')
    actual = actual.sort_values(by =['acc_id'], axis = 0) # 실제 답안을 acc_id 기준으로 정렬
    actual =actual.reset_index(drop=True)
    
    if predict.acc_id.equals(actual.acc_id) == False:
        print('acc_id of predicted and actual label does not match')
        sys.exit() # 예측 답안의 acc_id와 실제 답안의 acc_id가 다른 경우 에러처리 
    else:
            
        S, alpha, L, sigma = 30, 0.01, 0.1, 15  
        cost, gamma, add_rev = 0,0,0 
        profit_result = []
        survival_time_pred = list(predict.survival_time)
        amount_spent_pred = list(predict.amount_spent)
        survival_time_actual = list(actual.survival_time)
        amount_spent_actual = list(actual.amount_spent)    
        for i in range(len(survival_time_pred)):
            if survival_time_pred[i] == 64 :                 
                cost = 0
                optimal_cost = 0
            else:
                cost = alpha * S * amount_spent_pred[i]                    #비용 계산
                optimal_cost = alpha * S * amount_spent_actual[i]          #적정비용 계산 
            
            if optimal_cost == 0:
                gamma = 0
            elif cost / optimal_cost < L:
                gamma = 0
            elif cost / optimal_cost >= 1:
                gamma = 1
            else:
                gamma = (cost)/((1-L)*optimal_cost) - L/(1-L)              #반응률 계산
            
            if survival_time_pred[i] == 64 or survival_time_actual[i] == 64:
                T_k = 0
            else:
                T_k = S * np.exp(-((survival_time_pred[i] - survival_time_actual[i])**2)/(2*(sigma)**2))    #추가 생존기간 계산
                
            add_rev = T_k * amount_spent_actual[i]                         #잔존가치 계산
    
           
            profit = gamma * add_rev - cost                                #유저별 기대이익 계산
            profit_result.append(profit)
            
        score = sum(profit_result)                                         #기대이익 총합 계산
        # print(score)
    return score



In [0]:
valid_actual = valid_dfl[['acc_id', 'survival_time', 'amount_spent']]

In [0]:
valid_predict = pd.DataFrame({'acc_id':valid_dfla['acc_id'], 'survival_time':pred_s, 'amount_spent':pred_a})

In [34]:
score_function(valid_predict, valid_actual)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


717.7765122822155

## TEST

In [0]:
test1_activity = pd.read_csv(path+'test/'+'test1_activity.csv')
test2_activity = pd.read_csv(path+'test/'+'test2_activity.csv')

In [0]:
test1_activity['game_money_change'] = np.abs(test1_activity['game_money_change'])
test1_activity_group = test1_activity.groupby('acc_id').agg({'day':'nunique', 
                                                             'char_id':'nunique', 
                                                             'server':'nunique', 
                                                             'playtime':['sum','mean'], 
                                                             'npc_kill':['sum','mean'], 
                                                             'solo_exp':'sum',
                                                             'party_exp':'sum', 
                                                             'quest_exp':'sum',
                                                             'rich_monster':'sum', 
                                                             'death':'sum', 
                                                             'revive':'sum',
                                                             'exp_recovery':'sum',
                                                             'fishing':'sum',
                                                             'private_shop':'sum',
                                                             'game_money_change':'sum',
                                                             'enchant_count':'sum'})

test2_activity['game_money_change'] = np.abs(test2_activity['game_money_change'])
test2_activity_group = test2_activity.groupby('acc_id').agg({'day':'nunique', 
                                                             'char_id':'nunique', 
                                                             'server':'nunique', 
                                                             'playtime':['sum','mean'], 
                                                             'npc_kill':['sum','mean'], 
                                                             'solo_exp':'sum',
                                                             'party_exp':'sum', 
                                                             'quest_exp':'sum',
                                                             'rich_monster':'sum', 
                                                             'death':'sum', 
                                                             'revive':'sum',
                                                             'exp_recovery':'sum',
                                                             'fishing':'sum',
                                                             'private_shop':'sum',
                                                             'game_money_change':'sum',
                                                             'enchant_count':'sum'})

In [0]:
test1_activity_group.columns = list(map('_'.join, test1_activity_group.columns.values))
test2_activity_group.columns = list(map('_'.join, test2_activity_group.columns.values))

In [0]:
test1_activity_grouped = test1_activity_group.reset_index()
test2_activity_grouped = test2_activity_group.reset_index()

In [39]:
(test1_activity_grouped.columns,test2_activity_grouped.columns) 

(Index(['acc_id', 'day_nunique', 'char_id_nunique', 'server_nunique',
        'playtime_sum', 'playtime_mean', 'npc_kill_sum', 'npc_kill_mean',
        'solo_exp_sum', 'party_exp_sum', 'quest_exp_sum', 'rich_monster_sum',
        'death_sum', 'revive_sum', 'exp_recovery_sum', 'fishing_sum',
        'private_shop_sum', 'game_money_change_sum', 'enchant_count_sum'],
       dtype='object'),
 Index(['acc_id', 'day_nunique', 'char_id_nunique', 'server_nunique',
        'playtime_sum', 'playtime_mean', 'npc_kill_sum', 'npc_kill_mean',
        'solo_exp_sum', 'party_exp_sum', 'quest_exp_sum', 'rich_monster_sum',
        'death_sum', 'revive_sum', 'exp_recovery_sum', 'fishing_sum',
        'private_shop_sum', 'game_money_change_sum', 'enchant_count_sum'],
       dtype='object'))

In [40]:
test1_activity_grouped.head()

,acc_id,day_nunique,char_id_nunique,server_nunique,playtime_sum,playtime_mean,npc_kill_sum,npc_kill_mean,solo_exp_sum,party_exp_sum,quest_exp_sum,rich_monster_sum,death_sum,revive_sum,exp_recovery_sum,fishing_sum,private_shop_sum,game_money_change_sum,enchant_count_sum
0,7,10,3,2,1.252251,0.104354,0.719474,0.059956,8.624290,0.000000,34.556743,0,0.737650,0.494674,0.0,0.000000,0.000000,0.001291,0.0
1,15,28,3,3,85.705431,1.032596,18.207875,0.219372,0.630317,0.000000,0.000000,0,7.868271,6.183422,0.0,0.000000,0.000000,0.023029,0.0
2,16,3,7,1,0.081923,0.009103,0.000000,0.000000,0.000000,0.000000,0.100846,0,0.000000,0.000000,0.0,0.000000,0.000000,0.034526,0.0
3,18,13,1,1,40.008820,3.077602,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.0,0.000000,36.945877,0.921976,0.0
4,19,28,4,4,84.139533,2.052184,79.866399,1.947961,0.271431,0.954236,0.007157,2,0.737650,0.494674,0.0,37.465503,0.000000,1.863396,0.0


In [41]:
test2_activity_grouped.head()

,acc_id,day_nunique,char_id_nunique,server_nunique,playtime_sum,playtime_mean,npc_kill_sum,npc_kill_mean,solo_exp_sum,party_exp_sum,quest_exp_sum,rich_monster_sum,death_sum,revive_sum,exp_recovery_sum,fishing_sum,private_shop_sum,game_money_change_sum,enchant_count_sum
0,1,28,2,1,88.551668,2.012538,58.285882,1.324679,0.512386,0.000000,0.000000,0,2.704718,0.000000,0.0,0.000000,0.0,0.003433,0.0
1,3,21,9,5,2.694094,0.081639,0.092596,0.002806,0.010214,0.000000,0.078912,2,0.000000,0.000000,0.0,1.025441,0.0,0.051625,0.0
2,9,4,4,1,0.081923,0.016385,0.015883,0.003177,0.005888,0.000000,0.066681,0,0.000000,0.000000,0.0,0.000000,0.0,0.028409,0.0
3,14,12,8,2,6.558516,0.142576,0.215944,0.004694,0.033683,0.000837,0.006073,4,0.983534,0.989348,0.0,0.526569,0.0,0.052707,0.0
4,26,28,6,4,31.816526,0.435843,3.774791,0.051709,0.428995,0.009611,0.000868,5,29.506016,29.680428,0.0,83.157199,0.0,5.968221,0.0


In [0]:
class model(object):
    def __init__(self, train, test, folds=FOLDS, seed=SEED, log=True):
        self.train = train
        self.test = test
        self.kf = KFold(n_splits=folds, random_state=seed)
        self.log = log
    
    def ridge_model(self):
        oof = np.zeros(len(self.train))
        pred = np.zeros(len(self.test))
        for trn_idx, val_idx in self.kf.split(self.train):
            train_df = self.train.loc[trn_idx]
            valid_df = self.train.loc[val_idx].drop(lalabel, axis=1)
            if self.log:
                ridge_model = Ridge().fit(train_df.drop(lalabel, axis=1), np.log1p(train_df[lalabel]))
            else:
                ridge_model = Ridge().fit(train_df.drop(lalabel, axis=1), train_df[lalabel])
            oof[val_idx] = ridge_model.predict(valid_df)
            pred += ridge_model.predict(self.test)/self.kf.n_splits
        return oof, pred
    
    
    def rf_model(self):
        
    
        
        return oof, pred
    
    
    def xgb_model(self):
        warnings.filterwarnings(action='ignore')
        
        params={
            'eta': 0.001,
            'max_depth': 16,
            'min_child_weight': 16,
            'n_estimators' : 10000,
            'subsample': 0.9
        }
    
    
        oof = np.zeros(len(self.train))
        pred = np.zeros(len(self.test))
        for trn_idx, val_idx in self.kf.split(self.train):
            if self.log:
                train_df = xgb.DMatrix(self.train.loc[trn_idx].drop(lalabel, axis=1), label=np.log1p(self.train.loc[trn_idx, lalabel]))
                valid_df = xgb.DMatrix(self.train.loc[val_idx].drop(lalabel, axis=1), label=np.log1p(self.train.loc[val_idx, lalabel]))
            else:
                train_df = xgb.DMatrix(self.train.loc[trn_idx].drop(lalabel, axis=1), label=self.train.loc[trn_idx, lalabel])
                valid_df = xgb.DMatrix(self.train.loc[val_idx].drop(lalabel, axis=1), label=self.train.loc[val_idx, lalabel])
            xgb_model = xgb.train(params, train_df, num_boost_round=30000, evals=[(train_df, 'train'), (valid_df, 'val')], verbose_eval=5000, early_stopping_rounds=500)
            oof[val_idx] = xgb_model.predict(xgb.DMatrix(self.train.loc[val_idx].drop(lalabel, axis=1)))
            pred += xgb_model.predict(xgb.DMatrix(self.test))/self.kf.n_splits
        return oof, pred
    
    def rmse(self, true, pred):
        
        if self.log:
            true = np.expm1(pred)
            mse = mean_squared_error(true, pred)
            rmse = np.round(np.sqrt(mse), 2)
        else:
            mse = mean_squared_error(true, pred)
            rmse = np.round(np.sqrt(mse), 2)
    
        return rmse

In [0]:
lalabel = lalabels[0]

In [131]:
mdl = model(train_dfls, test1_activity_grouped)
mdl

In [132]:
test1_activity_grouped.shape

(20000, 19)

In [133]:
mdl.ridge_model()[1].shape

(20000,)

In [0]:
test1_pred_s = mdl.ridge_model()[1]

In [0]:
lalabel = lalabels[1]

In [136]:
mdl = model(train_dfla, test1_activity_grouped)
mdl

In [137]:
mdl.ridge_model()[1].shape

(20000,)

In [0]:
test1_pred_a = mdl.ridge_model()[1]

In [0]:
test1_predict = pd.DataFrame({'acc_id':test1_activity_grouped['acc_id'], 'survival_time':np.round(test1_pred_s), 'amount_spent':test1_pred_a})

In [0]:
save_path = "../gdrive/My Drive/bigcontest2019/metrics/hh_inference/0820/"

In [0]:
test1_predict.to_csv(save_path + 'test1_predict.csv', index=False, encoding="utf-8")

In [0]:
lalabel = lalabels[0]

In [143]:
mdl = model(train_dfls, test2_activity_grouped)
mdl

In [144]:
test2_activity_grouped.shape

(20000, 19)

In [145]:
mdl.ridge_model()[1].shape

(20000,)

In [0]:
test2_pred_s = mdl.ridge_model()[1]

In [0]:
lalabel = lalabels[1]

In [148]:
mdl = model(train_dfla, test2_activity_grouped)
mdl

In [149]:
mdl.ridge_model()[1].shape

(20000,)

In [0]:
test2_pred_a = mdl.ridge_model()[1]

In [0]:
test2_predict = pd.DataFrame({'acc_id':test2_activity_grouped['acc_id'], 'survival_time':np.round(test2_pred_s), 'amount_spent':test2_pred_a})

In [0]:
save_path = "../gdrive/My Drive/bigcontest2019/metrics/hh_inference/0820/"

In [0]:
test2_predict.to_csv(save_path + 'test2_predict.csv', index=False, encoding="utf-8")

| 변경 시도 사항 | validation score | test1 score | test2 score |
| -------------  | ---------------- | ------------| ------------|
|기본 | 0.000    |                  |             |             |

In [154]:
!ls "../gdrive/My Drive/bigcontest2019/metrics/hh_inference/0820/"

test1_predict.csv  test2_predict.csv


In [155]:
np.round(test1_pred_s)

array([3., 3., 2., ..., 4., 4., 3.])